# HAND GESTURE-BASED INTERACTION

---

Group members:
*   Ada Yılmaz
*   Ceren Şahin
*   Sima Adleyba
*   Selen Naz Gürsoy

### Installing necessary libraries and models

In [ ]:
#install mediapipe
%pip install -q mediapipe

In [ ]:
#download a model that can recognize 7 hand gestures: 👍, 👎, ✌️, ☝️, ✊, 👋, 🤟
!wget -q https://storage.googleapis.com/mediapipe-models/gesture_recognizer/gesture_recognizer/float16/1/gesture_recognizer.task

In [ ]:
#download test images from pixabay
import urllib

IMAGE_FILENAMES = ['thumbs_down.jpg', 'victory.jpg', 'thumbs_up.jpg', 'pointing_up.jpg']

for name in IMAGE_FILENAMES:
  url = f'https://storage.googleapis.com/mediapipe-tasks/gesture_recognizer/{name}'
  urllib.request.urlretrieve(url, name)

In [ ]:
#or we can use our own images as shown below

# from google.colab import files
# uploaded = files.upload()

# for filename in uploaded:
#   content = uploaded[filename]
#   with open(filename, 'wb') as f:
#     f.write(content)
# IMAGE_FILENAMES = list(uploaded.keys())

# print('Uploaded files:', IMAGE_FILENAMES)

### Functions for visualization

In [1]:
from matplotlib import pyplot as plt
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

In [2]:
#some functions to visualize the gesture recognition results.
import math

plt.rcParams.update({
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.spines.left': False,
    'axes.spines.bottom': False,
    'xtick.labelbottom': False,
    'xtick.bottom': False,
    'ytick.labelleft': False,
    'ytick.left': False,
    'xtick.labeltop': False,
    'xtick.top': False,
    'ytick.labelright': False,
    'ytick.right': False
})

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles


def display_one_image(image, title, subplot, titlesize=16):
    """Displays one image along with the predicted category name and score."""
    plt.subplot(*subplot)
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize), color='black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)


def display_batch_of_images_with_gestures_and_hand_landmarks(images, results):
    """Displays a batch of images with the gesture category and its score along with the hand landmarks."""
    # Images and labels.
    images = [image.numpy_view() for image in images]
    gestures = [top_gesture for (top_gesture, _) in results]
    multi_hand_landmarks_list = [multi_hand_landmarks for (_, multi_hand_landmarks) in results]

    # Auto-squaring: this will drop data that does not fit into square or square-ish rectangle.
    rows = int(math.sqrt(len(images)))
    cols = len(images) // rows

    # Size and spacing.
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols, 1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))

    # Display gestures and hand landmarks.
    for i, (image, gestures) in enumerate(zip(images[:rows*cols], gestures[:rows*cols])):
        title = f"{gestures.category_name} ({gestures.score:.2f})"
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols) * 40 + 3
        annotated_image = image.copy()

        for hand_landmarks in multi_hand_landmarks_list[i]:
          hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
          hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
          ])

          mp_drawing.draw_landmarks(
            annotated_image,
            hand_landmarks_proto,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

        subplot = display_one_image(annotated_image, title, subplot, titlesize=dynamic_titlesize)

    # Layout.
    plt.tight_layout()
    plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()

### Preview the images

In [3]:
import cv2
import math

DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480

def resize_and_show(image, name):
    h, w = image.shape[:2]
    if h < w:
        img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h / (w / DESIRED_WIDTH))))
    else:
        img = cv2.resize(image, (math.floor(w / (h / DESIRED_HEIGHT)), DESIRED_HEIGHT))
    
    # Display the image in a window with a name
    cv2.imshow(name, img)
    cv2.waitKey(0)  # Wait for a key press to close the window
    cv2.destroyAllWindows()  # Close the window after key press

# Example usage
images = {name: cv2.imread(name) for name in IMAGE_FILENAMES}

for name, image in images.items():
    if image is not None:
        print(f"Displaying: {name}")
        resize_and_show(image, name)
    else:
        print(f"Error: Could not read image {name}")


NameError: name 'IMAGE_FILENAMES' is not defined

### Google implementation - do not run
This first one is a how to from google, we can create our own by following the steps given.


In [ ]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an GestureRecognizer object.
base_options = python.BaseOptions(model_asset_path='gesture_recognizer.task')
options = vision.GestureRecognizerOptions(base_options=base_options)
recognizer = vision.GestureRecognizer.create_from_options(options)

images = []
results = []
for image_file_name in IMAGE_FILENAMES:
    
  # STEP 3: Load the input image.
  image = mp.Image.create_from_file(image_file_name)

  # STEP 4: Recognize gestures in the input image.
  recognition_result = recognizer.recognize(image)

  # STEP 5: Process the result. In this case, visualize it.
  images.append(image)
  top_gesture = recognition_result.gestures[0][0]
  hand_landmarks = recognition_result.hand_landmarks
  results.append((top_gesture, hand_landmarks))

display_batch_of_images_with_gestures_and_hand_landmarks(images, results)

### Example - do not run
Tried an example to see how it works and update the functions accordingly

In [ ]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

def detect_peace_sign(hand_landmarks):
    # Get coordinates of relevant landmarks
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
    
    
    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
    ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]
    pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]


    # Define peace sign gesture: index and middle fingers up, ring and pinky fingers down
    if (
        # Index and middle fingers' tips must be highest parts of those fingers
        (index_tip.y < index_mcp.y) and (middle_tip.y < middle_mcp.y)
        and
        # Index finger's tips must be higher than other fingers' mcp
        (index_tip.y < ring_mcp.y) and (index_tip.y < pinky_mcp.y)
        and
        # Middle finger's tips must be higher than other fingers' mcp
        (middle_tip.y < ring_mcp.y) and (middle_tip.y < pinky_mcp.y)
    ):
        return True
    return False


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Detect custom gesture (peace sign)
            if detect_peace_sign(hand_landmarks):
                cv2.putText(frame, 'Peace Sign Detected!', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                # Trigger interaction (e.g., print a message)
                print("Peace sign gesture detected!")

    # Display the frame
    cv2.imshow('MediaPipe Hands', frame)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

### Gesture Detection Functions
After checking how the upper ones worked, we implemented fixed, more robust versions of gesture detection functions.

In [ ]:
import time

# Global variables for tracking gestures and cooldown

# The currently detected gesture
current_gesture = None

# Time when the current gesture expires
gesture_reset_time = 0

# Cooldown variables for scrolling
previous_thumb_tip = None
previous_index_tip = None
last_gesture_time = 0
COOLDOWN_PERIOD = 1.5

# Reset current gesture when it expires
def reset_gesture():
    global current_gesture, gesture_reset_time
    if time.time() > gesture_reset_time:
        current_gesture = None

# Gesture detection functions
def detect_peace_sign(hand_landmarks):
    
    # Track the time of the last detected gesture
    global last_gesture_time

    # Get landmarks
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]

    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
    ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]
    pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]

    # Check that index and middle are raised above their MCPs and other MCPs
    index_and_middle_up = (
        (index_tip.y < index_mcp.y) and
        (middle_tip.y < middle_mcp.y) and
        (index_tip.y < ring_mcp.y) and 
        (index_tip.y < pinky_mcp.y) and
        (middle_tip.y < ring_mcp.y) and 
        (middle_tip.y < pinky_mcp.y)
    )

    # Check spacing between index and middle fingers
    index_middle_spacing = abs(index_tip.x - middle_tip.x) > 0.1

    # Check that ring and pinky are down (their tips should be below their MCP joints)
    ring_and_pinky_down = (
        (ring_tip.y > ring_mcp.y + 0.02) and
        (pinky_tip.y > pinky_mcp.y + 0.02)
    )

    if index_and_middle_up and index_middle_spacing and ring_and_pinky_down:
        
        # Update last gesture time (to apply cooldown for peace sign gesture)
        last_gesture_time = time.time()
        return True
    return False



def detect_thumbs_up(hand_landmarks, margin=0.05):
    
    # Track the time of the last detected gesture
    global last_gesture_time 

    # Get landmarks
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
    ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]
    pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]
    thumb_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP]
    
    thumb_base = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_CMC]

    # Thumb tip should be above other fingertips
    thumb_tip_up = ((thumb_tip.y + margin < index_tip.y) and
                    (thumb_tip.y + margin < middle_tip.y) and
                    (thumb_tip.y + margin < ring_tip.y) and
                    (thumb_tip.y + margin < pinky_tip.y) and
                    (thumb_tip.y < thumb_mcp.y))
    
    # Other fingers should be in order from top to bottom
    other_fingers_ordered = ((index_mcp.y < middle_mcp.y) and
                             (middle_mcp.y < ring_mcp.y) and
                             (ring_mcp.y < pinky_mcp.y))
    
    
    if thumb_tip_up and other_fingers_ordered:
        
        # Update last gesture time (to apply cooldown for thumbs-up gesture)
        last_gesture_time = time.time()
        return True
    return False


def detect_thumbs_down(hand_landmarks, margin=0.05):
    
    # Track the time of the last detected gesture
    global last_gesture_time

    # Get landmarks
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
    ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]
    pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]

    # Thumb tip must be lower than anything else
    thumb_tip_down = ((thumb_tip.y > index_tip.y + margin) and
                    (thumb_tip.y > middle_tip.y + margin) and
                    (thumb_tip.y > ring_tip.y + margin) and
                    (thumb_tip.y > pinky_tip.y + margin))
    
    # Other fingers should be in the order (from top to down) pinky > ring > middle > index finger
    other_fingers_ordered = ((index_mcp.y > middle_mcp.y) and
                             (middle_mcp.y > ring_mcp.y) and
                             (ring_mcp.y > pinky_mcp.y))
    
    if thumb_tip_down and other_fingers_ordered:
        
        # Update last gesture time (to apply cooldown for thumbs-down gesture)
        last_gesture_time = time.time()
        return True
    return False

def detect_scroll(hand_landmarks, threshold=0.1, dominance_ratio=4.0):
    
    # Get previous positions and last gesture time
    global previous_thumb_tip, previous_index_tip, last_gesture_time

    # Get current positions
    current_thumb_tip_x = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x
    current_thumb_tip_y = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y
    current_index_tip_x = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
    current_index_tip_y = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y

    # Check for cooldown
    current_time = time.time()
    
    # If last gesture happened not before cooldown, return False (no scroll)
    if current_time - last_gesture_time < COOLDOWN_PERIOD:
        return False, None 

    # Initialize previous positions if not set
    if previous_thumb_tip is None or previous_index_tip is None:
        previous_thumb_tip = (current_thumb_tip_x, current_thumb_tip_y)
        previous_index_tip = (current_index_tip_x, current_index_tip_y)
        return False, None

    # Calculate location changes
    thumb_horizontal_disp = current_thumb_tip_x - previous_thumb_tip[0]
    thumb_vertical_disp = current_thumb_tip_y - previous_thumb_tip[1]
    index_horizontal_disp = current_index_tip_x - previous_index_tip[0]
    index_vertical_disp = current_index_tip_y - previous_index_tip[1]

    # Average the movements of thumb and index for robustness
    horizontal_disp = (thumb_horizontal_disp + index_horizontal_disp) / 2
    vertical_disp = (thumb_vertical_disp + index_vertical_disp) / 2

    # Determine dominant movement (we want to return only a horizontal or vertical movement)
    horizontal_movement = abs(horizontal_disp) > threshold
    vertical_movement = abs(vertical_disp) > threshold

    # Check dominance
    if horizontal_movement and abs(horizontal_disp) > dominance_ratio * abs(vertical_disp):
        direction = "right" if horizontal_disp > 0 else "left"
        dominant_axis = "horizontal"
    elif vertical_movement and abs(vertical_disp) > dominance_ratio * abs(horizontal_disp):
        direction = "down" if vertical_disp > 0 else "up"
        dominant_axis = "vertical"
    else:
        direction = None
        dominant_axis = None

    # If there is a dominant movement
    if dominant_axis:
        
        # Update positions
        previous_thumb_tip = (current_thumb_tip_x, current_thumb_tip_y)
        previous_index_tip = (current_index_tip_x, current_index_tip_y)
        
        # Update last gesture time
        last_gesture_time = current_time 
        return True, direction

    return False, None

In [ ]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Global variables for tracking previous positions
previous_thumb_tip = None
previous_index_tip = None

print("Press 'Esc' to exit.")

# Initialize gesture display variables
current_gesture = None 
gesture_display_time = 0 
GESTURE_DISPLAY_DURATION = 1.5 

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Detect gestures
            detected_gesture = None
            if detect_peace_sign(hand_landmarks):
                detected_gesture = "Peace Sign Detected!"
            elif detect_thumbs_up(hand_landmarks):
                detected_gesture = "Thumbs Up Detected!"
            elif detect_thumbs_down(hand_landmarks):
                detected_gesture = "Thumbs Down Detected!"
            
            # If a gesture is detected, update the display variables
            if detected_gesture:
                current_gesture = detected_gesture
                gesture_display_time = time.time()
            else:
                # Only check for scroll if no other gesture is detected
                detected, direction = detect_scroll(hand_landmarks, threshold=0.1, dominance_ratio=4.0)
                if detected:
                    current_gesture = f"Scroll Detected: {direction.title()}!"
                    gesture_display_time = time.time()

    # Display the current gesture if within the display duration
    if current_gesture and (time.time() - gesture_display_time < GESTURE_DISPLAY_DURATION):
        cv2.putText(frame, current_gesture, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Gesture Recognition', frame)

    # Break the loop when 'Esc' key is pressed
    if cv2.waitKey(5) & 0xFF == 27:
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

### Cursor control

In [ ]:
import cv2
import mediapipe as mp
import pyautogui

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Get screen dimensions
screen_width, screen_height = pyautogui.size()

def is_click_gesture(hand_landmarks):
    """Detect a pinching gesture for a left click."""
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

    # Calculate the 3D Euclidean distance between index tip and thumb tip
    distance = ((index_tip.x - thumb_tip.x) ** 2 +
                (index_tip.y - thumb_tip.y) ** 2 +
                (index_tip.z - thumb_tip.z) ** 2) ** 0.5

    # Adjust threshold based on typical 3D distances observed
    return distance < 0.05


# Open webcam
cap = cv2.VideoCapture(0)

print("Press 'Esc' to exit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for a mirror-like effect
    frame = cv2.flip(frame, 1)

    # Get frame dimensions
    frame_height, frame_width, _ = frame.shape

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get index finger tip coordinates
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

            # Normalize coordinates to screen dimensions
            cursor_x = int(index_finger_tip.x * screen_width)
            cursor_y = int(index_finger_tip.y * screen_height)

            # Move the mouse cursor
            pyautogui.moveTo(cursor_x, cursor_y)

            # Detect click gesture
            if is_click_gesture(hand_landmarks):
                pyautogui.click()
                cv2.putText(frame, "Click!", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Gesture-Based Cursor Control", frame)

    # Break the loop when 'Esc' key is pressed
    if cv2.waitKey(5) & 0xFF == 27:
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import mediapipe as mp
import tkinter as tk
from PIL import Image, ImageTk, ImageDraw, ImageFont
import os
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Initialize the main Tkinter window
root = tk.Tk()
root.title("Gesture-Controlled Instagram Feed")
root.geometry("1024x768")
root.configure(bg="#f0f0f0")

# Load photos into a list
photo_folder = "Images/"
photo_files = [os.path.join(photo_folder, f) for f in os.listdir(photo_folder) if f.endswith((".jpg", ".png"))]
photos = [Image.open(photo).resize((400, 400)) for photo in photo_files]

# Create a label to display a single photo
photo_label = tk.Label(root, bg="#ffffff", width=400, height=400)
photo_label.pack(side=tk.LEFT, padx=10, pady=10)

# Feedback label
feedback_label = tk.Label(root, text="Perform gestures to interact!", font=("Helvetica", 14), bg="#f0f0f0")
feedback_label.pack(side=tk.BOTTOM, pady=20)

# Interaction states
current_photo_index = 0  # Start with the first photo
liked_photos = {}
disliked_photos = {}
saved_photos = []  # List to store saved photos
showing_saved_photos = False  # Track whether we are showing saved photos

# Camera feed frame
camera_frame = tk.Label(root, bg="#000000", width=500, height=700)
camera_frame.pack(side=tk.RIGHT, padx=10, pady=10)

# Function to add emoji to photos
def add_emoji(photo, emoji):
    """Add an emoji to the given photo."""
    img = photo.copy()
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype("arial.ttf", 50)  # Ensure you have a compatible font installed
    draw.text((150, 150), emoji, fill="red", font=font)
    return img

# Function to update the displayed photo
def update_photo():
    """Update the currently displayed photo based on the index."""
    global current_photo_index
    if showing_saved_photos:
        if len(saved_photos) == 0:
            feedback_label.config(text="No saved photos available!")
            photo_label.config(image="")
            return
        if 0 <= current_photo_index < len(saved_photos):
            img = ImageTk.PhotoImage(saved_photos[current_photo_index])
            photo_label.configure(image=img)
            photo_label.img = img  # Keep a reference to avoid garbage collection
    else:
        if 0 <= current_photo_index < len(photos):
            current_photo = photos[current_photo_index]
            if current_photo_index in liked_photos:
                current_photo = liked_photos[current_photo_index]
            elif current_photo_index in disliked_photos:
                current_photo = disliked_photos[current_photo_index]
            img = ImageTk.PhotoImage(current_photo)
            photo_label.configure(image=img)
            photo_label.img = img  # Keep a reference to avoid garbage collection

# Function to toggle between all photos and saved photos
def show_saved_photos():
    global showing_saved_photos, current_photo_index
    showing_saved_photos = not showing_saved_photos
    current_photo_index = 0  # Reset to the first photo
    if showing_saved_photos:
        feedback_label.config(text="Showing Saved Photos 📂")
    else:
        feedback_label.config(text="Showing All Photos 🌍")
    update_photo()

# Gesture Detection Logic
# Gesture Detection Logic
def detect_gesture(hand_landmarks):
    """Detect gestures for liking, disliking, saving, scrolling, and clicking."""
    global current_photo_index, current_gesture, gesture_reset_time, last_gesture_time

    # Reset the current gesture if its reset time has passed
    reset_gesture()

    # Cooldown check to avoid repeated gesture detection within the cooldown period
    current_time = time.time()
    if current_time - last_gesture_time < COOLDOWN_PERIOD:
        return

    # Detect gestures
    if detect_thumbs_up(hand_landmarks):
        current_gesture = "like"
        gesture_reset_time = time.time() + COOLDOWN_PERIOD
        feedback_label.config(text="Photo Liked! ❤️")
        if current_photo_index not in liked_photos:
            liked_photos[current_photo_index] = add_emoji(photos[current_photo_index], "❤️")
        update_photo()
        return

    if detect_thumbs_down(hand_landmarks):
        current_gesture = "dislike"
        gesture_reset_time = time.time() + COOLDOWN_PERIOD
        feedback_label.config(text="Photo Disliked! 👎")
        if current_photo_index not in disliked_photos:
            disliked_photos[current_photo_index] = add_emoji(photos[current_photo_index], "👎")
        update_photo()
        return

    if detect_peace_sign(hand_landmarks):
        current_gesture = "save"
        gesture_reset_time = time.time() + COOLDOWN_PERIOD
        feedback_label.config(text="Photo Saved! ✌️")
        current_photo = photos[current_photo_index]
        if current_photo not in saved_photos:
            saved_photos.append(current_photo)
        return

    # Detect scrolling
    scroll_detected, scroll_direction = detect_scroll(hand_landmarks)
    if scroll_detected:
        current_gesture = "scroll"
        gesture_reset_time = time.time() + COOLDOWN_PERIOD
        if scroll_direction == "up":
            feedback_label.config(text="Scrolling Up ⬆️")
            if current_photo_index > 0:
                current_photo_index -= 1
        elif scroll_direction == "down":
            feedback_label.config(text="Scrolling Down ⬇️")
            if showing_saved_photos:
                if current_photo_index < len(saved_photos) - 1:
                    current_photo_index += 1
            else:
                if current_photo_index < len(photos) - 1:
                    current_photo_index += 1
        update_photo()
        return

    # No gesture detected
    feedback_label.config(text="Perform gestures to interact!")


# Open webcam and process gestures
cap = cv2.VideoCapture(0)

def update_camera_feed():
    """Update the camera feed and process gestures in real-time."""
    global current_photo_index

    ret, frame = cap.read()
    if not ret:
        return

    # Flip the frame horizontally for a mirror-like effect
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB for MediaPipe
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame for hand landmarks
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks on the frame
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Detect gestures for liking, disliking, saving, scrolling, and clicking
            detect_gesture(hand_landmarks)

    # Convert the frame to an image for Tkinter
    frame_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    imgtk = ImageTk.PhotoImage(image=frame_image)

    # Display the camera feed in the GUI
    camera_frame.imgtk = imgtk
    camera_frame.configure(image=imgtk)

    # Schedule the next frame update
    root.after(10, update_camera_feed)

# Start with the first photo
update_photo()

# Add a button to toggle saved photos
saved_button = tk.Button(root, text="Show Saved Photos", command=show_saved_photos, font=("Helvetica", 12))
saved_button.pack(side=tk.BOTTOM, pady=10)

# Start the camera feed
update_camera_feed()

# Run the Tkinter event loop
root.mainloop()

# Release the camera
cap.release()


I0000 00:00:1733844366.579244   19642 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733844366.598994   21582 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733844366.615778   21581 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2024-12-10 18:26:15.746 Python[1529:19642] +[IMKClient subclass]: chose IMKClient_Legacy
2024-12-10 18:26:15.746 Python[1529:19642] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
W0000 00:00:1733844379.241659   21578 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
Exception in Tkinter callback
Traceback (most recent call last):
  File

# Instagram layout

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import requests
from io import BytesIO

class GestureApp:
    def __init__(self, root):
        self.root = root
        self.root.geometry("800x600")
        self.root.title("Gesture Controlled App")

        # Placeholder image
        self.image_index = 0
        self.images = self.fetch_images()  # Fetch Instagram images
        self.image_label = tk.Label(root)
        self.image_label.pack()

        # Buttons
        self.like_button = tk.Button(root, text="Like", command=self.like_picture)
        self.like_button.pack(side=tk.LEFT)

        self.dislike_button = tk.Button(root, text="Dislike", command=self.dislike_picture)
        self.dislike_button.pack(side=tk.LEFT)

        # Camera feed
        self.cap = cv2.VideoCapture(0)
        self.update_camera_feed()

    def fetch_images(self): #FILL IN FUNCTION
        images = []
        return images

    def update_camera_feed(self):
        ret, frame = self.cap.read()
        if not ret:
            print("Camera not working!")
            return

        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Process frame for hand landmarks (not shown here)

        # Convert the frame to an image format that Tkinter can use
        img = Image.fromarray(rgb_frame)
        imgtk = ImageTk.PhotoImage(image=img)
        self.image_label.imgtk = imgtk
        self.image_label.configure(image=imgtk)

        # Call this method again after 10 milliseconds
        self.root.after(10, self.update_camera_feed)

    def like_picture(self):
        print("Liked picture")

    def dislike_picture(self):
        print("Disliked picture")

# Create the Tkinter window and run the app
root = tk.Tk()
app = GestureApp(root)
root.mainloop()

# Instagram-like Interface Attempt

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
import os
import cv2
import random
import mediapipe as mp
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,  # Dynamic mode is faster
    max_num_hands=1,  # Detect only one hand to reduce processing
    min_detection_confidence=0.5,  # Slightly lower confidence threshold
    min_tracking_confidence=0.5
)

# Initialize Tkinter
root = tk.Tk()
root.geometry("1024x768")
root.title("Gesture-Controlled Instagram")

# Load Images
photo_folder = "Images/"  # Replace with your folder path
photo_files = [os.path.join(photo_folder, f) for f in os.listdir(photo_folder) if f.endswith((".jpg", ".png"))]
photos = [Image.open(photo).resize((400, 400)) for photo in photo_files]
random.shuffle(photos)  # Shuffle photos for randomness

# Initialize State
current_photo_index = 0
liked_photos = set()  # To store liked photo indices
disliked_photos = set()  # To store disliked photo indices

# Display Area
photo_label = tk.Label(root, width=400, height=400)
photo_label.pack(pady=20)

# Feedback Label
feedback_label = tk.Label(root, text="Perform gestures to interact!", font=("Helvetica", 14), bg="#f0f0f0")
feedback_label.pack(pady=10)

# Camera Feed Area
camera_frame = tk.Label(root, bg="black", width=500, height=500)
camera_frame.pack(pady=10)

# Functions
def update_photo():
    """Update the displayed photo based on the index."""
    if 0 <= current_photo_index < len(photos):
        img = ImageTk.PhotoImage(photos[current_photo_index])
        photo_label.configure(image=img)
        photo_label.image = img

def handle_like():
    """Handle 'like' action."""
    global current_photo_index
    liked_photos.add(current_photo_index)
    feedback_label.config(text="Liked ❤️")
    next_photo()

def handle_dislike():
    """Handle 'dislike' action."""
    global current_photo_index
    disliked_photos.add(current_photo_index)
    feedback_label.config(text="Disliked 👎")
    next_photo()

def next_photo():
    """Go to the next photo."""
    global current_photo_index
    current_photo_index += 1
    if current_photo_index >= len(photos):
        current_photo_index = 0  # Loop back to the start
    update_photo()

def scroll(direction):
    """Scroll through the photos."""
    global current_photo_index
    if direction == "up" and current_photo_index > 0:
        current_photo_index -= 1
    elif direction == "down" and current_photo_index < len(photos) - 1:
        current_photo_index += 1
    update_photo()

# Gesture Detection Integration
cap = cv2.VideoCapture(0)

def detect_gestures():
    """Integrate gesture detection with UI actions."""
    global cap, current_gesture

    try:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame")
            # Reschedule the detection after a short delay
            root.after(100, detect_gestures)
            return

        # Flip the frame horizontally for a mirror-like effect
        frame = cv2.flip(frame, 1)

        # Convert the frame to RGB for MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame for hand landmarks
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Add cooldown mechanism to prevent rapid gesture triggers
                current_time = time.time()
                if current_time - last_gesture_time > COOLDOWN_PERIOD:
                    # Detect gestures
                    if detect_thumbs_up(hand_landmarks):
                        handle_like()
                    elif detect_thumbs_down(hand_landmarks):
                        handle_dislike()
                    elif detect_peace_sign(hand_landmarks):
                        feedback_label.config(text="Photo Saved ✌️")
                    else:
                        detected, direction = detect_scroll(hand_landmarks)
                        if detected:
                            scroll(direction)

        # Convert the frame to an image for Tkinter
        frame_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        imgtk = ImageTk.PhotoImage(image=frame_image)

        # Display the camera feed in the GUI
        camera_frame.imgtk = imgtk
        camera_frame.configure(image=imgtk)

    except Exception as e:
        print(f"Error in gesture detection: {e}")

    # Always reschedule the next frame update
    root.after(10, detect_gestures)

# Initialize
update_photo()

# Start Gesture Detection
detect_gestures()

# Run Tkinter Event Loop
root.mainloop()

# Release Camera
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1733845170.356242   29315 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733845170.387841   31408 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733845170.404376   31408 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2024-12-10 18:39:36.133 Python[1670:29315] +[IMKClient subclass]: chose IMKClient_Legacy
2024-12-10 18:39:36.133 Python[1670:29315] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
W0000 00:00:1733845177.476484   31410 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detection: name 'mp_drawing' is not defined
Error in gesture detecti